<a href="https://colab.research.google.com/github/LizzyZhang-tutu/NLP_Learning/blob/master/SemanticSearchinDetails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
sentence.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [6]:
str.split(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [13]:
import numpy as np
token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))
print(','.join(vocab))
print(token_sequence)

26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
['Thomas', 'Jefferson', 'began', 'building', 'Monticello', 'at', 'the', 'age', 'of', '26.']


In [16]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)
print(num_tokens)
print(vocab_size)
# construct one hot vector
onehot_vector = np.zeros((num_tokens, vocab_size), int)
for i, word in enumerate(token_sequence):
  onehot_vector[i, vocab.index(word)] = 1

print(' '.join(vocab))
onehot_vector

10
10
26. Jefferson Monticello Thomas age at began building of the


array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [17]:
import pandas as pd
pd.DataFrame(onehot_vector, columns = vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


In [19]:
df = pd.DataFrame(onehot_vector, columns = vocab)
df[df == 0] = ''
df


,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,,,,1,,,,,,
1,,1,,,,,,,,
2,,,,,,,1,,,
3,,,,,,,,1,,
4,,,1,,,,,,,
5,,,,,,1,,,,
6,,,,,,,,,,1
7,,,,,1,,,,,
8,,,,,,,,,1,
9,1,,,,,,,,,


In [25]:
# we can use dictionary instead of a 2D array to save space
sentence_bow = {}
for token in sentence.split():
  sentence_bow[token] = 1
sorted(sentence_bow.items())

[('26.', 1),
 ('Jefferson', 1),
 ('Monticello', 1),
 ('Thomas', 1),
 ('age', 1),
 ('at', 1),
 ('began', 1),
 ('building', 1),
 ('of', 1),
 ('the', 1)]

In [29]:
# series is more efficient than dictionary
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent,1,1,1,1,1,1,1,1,1,1


In [35]:
sentences = "Thomas Jefferson began building Monticello at the age of 26.\n"
sentences += "Constructuion was done mostly by local masons and carpenters.\n"
sentences += "He moved into the South Pavilion in 1770.\n"
sentences += "Turning Monticello into a neoclassical masterpiece was Jefferson's obsession."

corpus = {}

for i, sent in enumerate(sentences.split('\n')):
  corpus['sent{}'.format(i)] = dict((token, 1) for token in sent.split())

df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df[df.columns[:10]]

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent0,1,1,1,1,1,1,1,1,1,1
sent1,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0


**Similarity using Dot Product**

In [36]:
v1 = np.array([1, 2, 3])
v2 = np.array([2, 3, 4])
v1.dot(v2)

20

In [37]:
(v1 * v2).sum()

20

In [38]:
sum([x1*x2 for x1, x2 in zip(v1, v2)])

20

In [39]:
df = df.T

print(df.sent0.dot(df.sent1))

0


In [40]:
print(df.sent0.dot(df.sent2))

1


In [42]:
print(df.sent0.dot(df.sent3))

1


In [44]:
# find shared word between sent0 and sent3
[(k, v) for (k, v) in (df.sent0 & df.sent3).items() if v]

[('Monticello', 1)]

**Reducing Dimensions of Vocabulary through token improvement**

In [2]:
import re
sentence = "Thomas Jefferson began building Monticello at the age of 26."
tokens = re.split(r'[-\s.,;!?]+', sentence)
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '']

**Reducing Dimensions of Vocabulary using n-grams**

One of the ways to maintain context information.

In [8]:
 # 1-agram
sentence = "Thomas Jefferson began building Monticello at the age of 26."
pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)
tokens = [x for x in tokens if x and x not in '- \t\n.,;!?']
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

In [9]:
#3-grams
from nltk.util import ngrams
list(ngrams(tokens, 3))


[('Thomas', 'Jefferson', 'began'),
 ('Jefferson', 'began', 'building'),
 ('began', 'building', 'Monticello'),
 ('building', 'Monticello', 'at'),
 ('Monticello', 'at', 'the'),
 ('at', 'the', 'age'),
 ('the', 'age', 'of'),
 ('age', 'of', '26')]

In [10]:
#2-grams
two_grams = list(ngrams(tokens, 2))
# joining every token as a string
[" ".join(x) for x in two_grams]

['Thomas Jefferson',
 'Jefferson began',
 'began building',
 'building Monticello',
 'Monticello at',
 'at the',
 'the age',
 'age of',
 'of 26']

Note that some n-grams a extremely rare. If tokens or n-grams are extremely rare, they don't carry any correlation with other words that you can use to help identify topics. So they are not useful for classification problems. Most 2-grams are pretty rare, even more so for three and four grams,because word combination are rarer than individual words. Your vocabulary size is exponentially approaching the number of n-grams in your corpus.

**Stop Words**

In [11]:
stop_words = ['a', 'the', 'an', 'on', 'of', 'off', 'this', 'is']
tokens = ['the', 'house', 'is', 'on', 'fire']
tokens_without_stopwords = [x for x in tokens if x not in stop_words]
print(tokens_without_stopwords)

['house', 'fire']


In [12]:
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
print(len(stop_words))
print(stop_words[:7])
print([sw for sw in stop_words if len(sw) == 1])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours']
['i', 'a', 's', 't', 'd', 'm', 'o', 'y']


In [13]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
print(len(sklearn_stop_words))
print(len(stop_words))
print(len(set(stop_words).union(sklearn_stop_words)))
print(len(set(stop_words).intersection(sklearn_stop_words))) # same stopwords in sklearn and nltk


318
179
378
119


Case Normalization (case folding)

> Normalizing word and character capitalization is one way to reduce your vocabulary size and generalize the NLP pipeline. It helps you consolidate words that are intended to mean the same thing under a single token.

> Often, CAPITALIZATION is to indicate a special noun, the name of a person or thing. If name and entity recognition is important in your pipeline, you may not want to normalize case. However, it means your vocabulary size is approximately twice as large.

> In some situations, cutting your vocabulary size by half can be worth the loss of information content.

